In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import pandas_profiling
%matplotlib inline

In [ ]:
df_train = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/train.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/test.csv')
submission = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv')

In [ ]:
print("train data have row {}, column {}.".format(df_train.shape[0], df_train.shape[1]))
print("test data have row {}, column {}.".format(df_test.shape[0], df_test.shape[1]))

In [ ]:
df_train.head()

In [ ]:
#there was no null value in the train data
df_train.info()

In [ ]:
# training data exploration
df_train.columns

In [ ]:
cat_feature = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7',
       'cat8', 'cat9']
num_feature = ['cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5',
       'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12',
       'cont13']

In [ ]:
# data distribution for continuous features
df_train.describe()

In [ ]:
#check target variable distribution
df_train['target'].hist(bins=30, figsize=(10,8))

In [ ]:
#check distribution for numerical features and all numerical features are mostly within the range (0,1)
df_train[num_feature].hist(bins = 30, figsize = (20, 15));

In [ ]:
# heatmap for numerical feature and some features are correlated with each other
corr = df_train[num_feature].corr()
plt.figure(figsize = (20,10))
sns.heatmap(corr, annot = True, cmap = 'coolwarm')

In [ ]:
# barplot for categoical feature
for cat in cat_feature:
    plt.figure(figsize = (10,6))
    sns.barplot(x = cat, y = 'target', data = df_train)
    plt.title('target vs.' + cat, fontsize = 30)
    plt.xlabel(cat, fontsize = 20)
    plt.ylabel('target', fontsize = 20)

In [ ]:
# data preparation for machine learning algorithm
# no missing values in train data
df_train.isnull().sum()

In [ ]:
df_train.shape

In [ ]:
df_train.head(2)

In [ ]:
# model training and evaluation
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor




In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
#function for comparing different approach
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators = 100, max_features = 10, random_state = 0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [ ]:
x = df_train.drop(['target', 'id'], axis =1)
y = df_train['target'].copy()

In [ ]:
#split validata set from training set
x_train, x_valid, y_train, y_valid = train_test_split(x,y, train_size =0.8, test_size = 0.2, random_state= 0)

In [ ]:
x_test = df_test.drop(['id'], axis =1).copy()

In [ ]:
label_X_train = x_train.copy()
label_X_valid = x_valid.copy()

label_encoder = LabelEncoder()
for col in cat_feature:
    label_X_train[col] = label_encoder.fit_transform(label_X_train[col])
    label_X_valid[col] = label_encoder.transform(label_X_valid[col])
    x_test[col] = label_encoder.transform(x_test[col])
 

In [ ]:
print("MAE (RandomForest):")
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

In [ ]:
model = RandomForestRegressor(n_estimators = 100, max_features = 10, random_state = 0)
model.fit(label_X_train, y_train)

In [ ]:
preds_test = model.predict(x_test)

In [ ]:
submission.head()

In [ ]:
# Save test predictions to file
output = pd.DataFrame({'id': df_test.id,
                       'target': preds_test})
output.to_csv('submission.csv', index = False)